In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [3]:
# My version with smaller chunks on safetensors for low RAM environments
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

In [4]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 8.4 MB/s eta 0:00:00


In [7]:
import torch
import transformers

pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [8]:
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:")

In [9]:
sequences

[{'generated_text': "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: Hah! Hi, Daniel! Long time no see!\nDaniel: I know! You's been out there exploring the universe, haven't you? \nGirafatron: Yeah! And I found some fascinating planets with some even more interesting species!\nDaniel: Wow, I'd love to hear all about them when you get back home.\nGirafatron: Well, until then, I'll just keep exploring the universe and learning about all the fascinating creatures out there!"}]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Copiando banco de dados do meu Drive para o colab local
!cp '/content/drive/MyDrive/ALLM/STORE_db.DB' '/content'

In [20]:
import sqlite3
import json

class CRUDSQLite:
    # DB_FILE = "DataSet/DataSet/STORE_db.DB"
    DB_FILE = "/content/STORE_db.DB"
    def __init__(self):
        self.conn = sqlite3.connect(self.DB_FILE)


    def get_user_by_id(self, id):
        query = '''
            SELECT * FROM customers_dataset WHERE customer_id == ?
        '''
        cursor = self.conn.execute(query, (id,))
        return cursor.fetchone()

    def get_all_customers(self):
        query = '''
            SELECT customer_id FROM customers_dataset
        '''
        cursor = self.conn.execute(query)
        return cursor.fetchall()

    def get_all_customers_jsonFormat(self):
        # query = '''
        #     SELECT customer_id FROM customers_dataset
        # '''

        query = '''
                SELECT

                        json_object(
                            'customer_id', customer_id
                            )

                FROM customers_dataset

        '''
        cursor = self.conn.execute(query)
        result = cursor.fetchall()

        if result:
            res = []
            if(len(result)>1):
                for item in result:
                    res.append(json.loads(item[0]))
            else:
                res.append(json.loads(result[0]))
            return res

        return result

    def get_order_by_id(self, id):
        query = '''
            SELECT * FROM orders_dataset WHERE order_id == ?
        '''
        cursor = self.conn.execute(query, (id,))
        return cursor.fetchone()

    def get_order_by_user_id(self, id):
        query = '''
            SELECT * FROM orders_dataset WHERE customer_id == ?;
        '''

        cursor = self.conn.execute(query, (id,))
        return cursor.fetchone()

    def get_order_by_user_id_jsonFormat(self, id):

        query = '''
                SELECT

                        json_object(
                            'order_id', order_id,
                            'customer_id', customer_id,
                            'order_status', order_status,
                            'order_purchase_timestamp', order_purchase_timestamp,
                            'order_approved_at', order_approved_at,
                            'order_delivered_carrier_date', order_delivered_carrier_date,
                            'order_delivered_customer_date', order_delivered_customer_date,
                            'order_estimated_delivery_date', order_estimated_delivery_date
                            )

                FROM orders_dataset WHERE customer_id == ?

        '''
        cursor = self.conn.execute(query, (id,))
        result = cursor.fetchone()

        if result:
            res = []
            if(len(result)>1):
                for item in result:
                    res.append(json.loads(item[0]))
            else:
                res.append(json.loads(result[0]))

            return res

        return result



    def get_orderItems_by_id(self, id):
        query = '''
            SELECT * FROM orders_items_dataset WHERE order_id == ?
        '''
        cursor = self.conn.execute(query, (id,))
        return cursor.fetchone()

    def get_product_by_id(self, id):
        query = '''
            SELECT * FROM products_dataset WHERE product_id == ?
        '''
        cursor = self.conn.execute(query, (id,))
        return cursor.fetchone()


    def close_connection(self):
        self.conn.close()

In [21]:
db = CRUDSQLite()

# Consulta pedidos pelo ID do usuario
result = db.get_order_by_user_id_jsonFormat("ffffe8b65bbe3087b653a978c870db99")


if result:
  print(result)
else:
  print("Registro não encontrado.")

db.close_connection()

[{'order_id': '2e935fa1d39497aa0ec3f1107fbfb5b8', 'customer_id': 'ffffe8b65bbe3087b653a978c870db99', 'order_status': 'unavailable', 'order_purchase_timestamp': '2017-09-29 14:07:03', 'order_approved_at': '2017-10-03 06:34:13', 'order_delivered_carrier_date': None, 'order_delivered_customer_date': None, 'order_estimated_delivery_date': '2017-10-24 00:00:00'}]


In [22]:
prompt = "context: "
for resultado_consulta in result:
  prompt = prompt + json.dumps(resultado_consulta) + ' \n '



In [23]:
consulta = "What is my order status?"

prompt = prompt + "Query: " + consulta



In [25]:
prompt

'context: {"order_id": "2e935fa1d39497aa0ec3f1107fbfb5b8", "customer_id": "ffffe8b65bbe3087b653a978c870db99", "order_status": "unavailable", "order_purchase_timestamp": "2017-09-29 14:07:03", "order_approved_at": "2017-10-03 06:34:13", "order_delivered_carrier_date": null, "order_delivered_customer_date": null, "order_estimated_delivery_date": "2017-10-24 00:00:00"} \n Query: What is my order status?'

In [24]:
pipeline(prompt)

[{'generated_text': 'context: {"order_id": "2e935fa1d39497aa0ec3f1107fbfb5b8", "customer_id": "ffffe8b65bbe3087b653a978c870db99", "order_status": "unavailable", "order_purchase_timestamp": "2017-09-29 14:07:03", "order_approved_at": "2017-10-03 06:34:13", "order_delivered_carrier_date": null, "order_delivered_customer_date": null, "order_estimated_delivery_date": "2017-10-24 00:00:00"} \n Query: What is my order status?\nBased on the provided query, the order status should be "unavailable". \n\nHowever, the query is missing a closing parenthesis in "unavailable". The query should be updated as follows: \n\nQuery: \nWhat is my order status?\n\nTo get the value of "order_id", the query should be enclosed in parentheses, so that it is treated as a subquery. \n\nThe query can be written as follows: \n\nQuery: \n\nWhat is my order status?\n\nThis will return true for all orders that are'}]